In [1]:
import pandas as pd
import numpy as np
import re 

from collections import Counter
import spacy #lemmatization 
#pre spacy treba v terminali doinstalovat 2 prikazy:
#pip3 install spacy
#python3 -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm") #nlp = spacy.load("en_core_web_lg") finalne

from sklearn.feature_extraction.text import CountVectorizer
from sklearn. feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import text 

In [35]:
data = pd.read_csv("data_1930_1990.csv", sep = '\t', header = None, names = ["id","year","title"] ,on_bad_lines='skip')
df = data[(data['year'] >= 1970) & (data['year'] <= 1990)]

In [36]:
data = (df.groupby([df['year']], group_keys=False)
         .apply(lambda x: x.sample(frac = 0.2)))

#df1 = (df.groupby([df['daate'].dt.year, df['daate'].dt.month], group_keys=False)
         #.apply(lambda x: x.sample(n=10)))

In [37]:
data.shape

(1130651, 3)

In [34]:
#kontrola ci sme vybtali z kazdeho roku 20%
#print(df.loc[df['year'] == 1987].shape)
#print(data.loc[data['year'] == 1987].shape)

In [38]:
data["title"] = data["title"].astype(str)  #zmenime stlpcec na typ str 
data['year'] = data['year'].astype('Int64')   #zmenime typ stlpca year na integer

In [39]:
#preprocessing text
def clean_text(text):
  text = re.sub(r'[^A-Za-z0-9 ]+',' ', text) #odstranime vsetko okrem pismen, cislic a medzier 
  return text
data["title_cleaned"] = data["title"].str.lower() #zmenime text na male pismena 
data['title_cleaned'] = data['title_cleaned'].apply(clean_text) #vycistime 

In [ ]:
df = data.copy(deep = True)

In [95]:
#tuto musime data rozdelit na 3 mensie dataframe po 400 000 zaznamov

data = df[:400000]
#data = df[400000:800000]
#data = df[800000:]

In [97]:
data["lemma"] = "" #new column - column title after lemmatization 

/var/folders/c2/k13xb47j4r13t6yhdv6xv15c0000gn/T/ipykernel_25914/2638881957.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["lemma"] = "" #new column - column title after lemmatization


In [98]:
#pridame stlpec lemmatization po jednotlivych vetach  

for index_row, row in data.iterrows():
    sentence_lemma = []
    #print(index_row, " ",row['title_cleaned'])
    data_tokens_spacy = nlp(row['title_cleaned'])
    for token in data_tokens_spacy:   
      sentence_lemma.append(token.lemma_)
    #print(sentence_lemma)
    data.at[index_row,"lemma"] = " ".join(sentence_lemma) 
    

In [100]:
#ulozime data do suboru oddelene tabulatorom

data.to_csv("1970/lemma_1970_90_p1.csv", sep = '\t', index = False)
#data.to_csv("1970/lemma_1970_90_p2.csv", sep = '\t', index = False)
#data.to_csv("1970/lemma_1970_90_p3.csv", sep = '\t', index = False)

In [101]:
#skombinujeme english a nase stopwords
stopwords_table = pd.read_csv('stop_words.txt')
custom_stopwords = list(stopwords_table['based'].values)
stop_words = text.ENGLISH_STOP_WORDS.union(custom_stopwords)

In [102]:
#pridame stlpec z lemma okrem stopwords
def rem_stop(text):
    return  ' '.join([word for word in text.split() if not word in stop_words])

#data['lemma'] = data['lemma'].astype(str)
data['lemma_without_stopwords'] = data['lemma'].apply(rem_stop)

/var/folders/c2/k13xb47j4r13t6yhdv6xv15c0000gn/T/ipykernel_25914/1768465911.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['lemma_without_stopwords'] = data['lemma'].apply(rem_stop)


In [103]:
#vytvorime count vectorizer a urobime vektor dat z lemma
cv = CountVectorizer (max_df=0.90,stop_words = stop_words, max_features=10000)
word_count_vector = cv.fit_transform(data['lemma'].values.astype('U'))

In [104]:
#vytvorime tfid model a nafitujeme data z countvectorizer
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [105]:
#funkcie na triedenie slov podla ich tfidf hodnot
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_keywords(feature_names, sorted_items, topn=10):
    sorted_items = sorted_items[:topn]
    feature_vals = []
    for idx, score in sorted_items:
        feature_vals.append(feature_names[idx])
    return feature_vals

In [106]:
#ziskame top 5 klucovych slov zo stlpca lemma s najvacsou tfidf hodnotou
idf_keywords = []
data['lemma'] = data['lemma'].values.astype('U')
feature_names = cv.get_feature_names_out()

for i in range(data['lemma'].size):
    tf_idf_vector = tfidf_transformer.transform(cv.transform([data.iloc[i]['lemma']]))
    sorted_items = sort_coo(tf_idf_vector.tocoo())
    keywords = extract_topn_keywords(feature_names, sorted_items, 5)
    #print("ahoj")
    idf_keywords.append(" ".join(keywords))

/var/folders/c2/k13xb47j4r13t6yhdv6xv15c0000gn/T/ipykernel_25914/846251361.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['lemma'] = data['lemma'].values.astype('U')


In [107]:
print(data.shape)
len(idf_keywords)
#mali by sa rovnat!

(330651, 6)


330651

In [108]:
#pridame stlpec s vybranymi keywords
data['tfidf_keywords'] = idf_keywords
data[:15]

/var/folders/c2/k13xb47j4r13t6yhdv6xv15c0000gn/T/ipykernel_25914/2242405160.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tfidf_keywords'] = idf_keywords


,id,year,title,title_cleaned,lemma,lemma_without_stopwords,tfidf_keywords
2661190,3701965,1986,Deferred consent. A new approach for resuscita...,deferred consent a new approach for resuscita...,defer consent a new approach for resuscitati...,defer consent resuscitation comatose,comatose consent resuscitation
1496842,2421014,1986,Intracavernous drug-induced erections in the m...,intracavernous drug induced erections in the m...,intracavernous drug induce erection in the man...,intracavernous drug induce erection male erect...,intracavernous erection erectile 100 dysfunction
2046176,3020756,1986,A small cell tumor.,a small cell tumor,a small cell tumor,small cell tumor,small tumor cell
2711807,3757207,1986,External quality assessment of assays of lead ...,external quality assessment of assays of lead ...,external quality assessment of assay of lead i...,external assay lead blood,external lead assay blood
2886521,3952360,1986,Technical aspects of interstitial hyperthermia.,technical aspects of interstitial hyperthermia,technical aspect of interstitial hyperthermia,technical aspect interstitial hyperthermia,technical interstitial hyperthermia aspect
2702922,3747280,1986,[A study on the course of the atrioventricular...,a study on the course of the atrioventricular...,a study on the course of the atrioventricula...,course atrioventricular conduction congenital ...,delineation electrophysiologic atrioventricula...
1909205,2871761,1986,"Effects of insulin, adenosine, and prostagland...",effects of insulin adenosine and prostagland...,effect of insulin adenosine and prostaglan...,insulin adenosine prostaglandin alpha adrenerg...,adipocyte respiration brown adenosine adrenergic
2523958,3548080,1986,[Measuring cyclosporin in immunosuppressive tr...,measuring cyclosporin in immunosuppressive tr...,measure cyclosporin in immunosuppressive tre...,measure cyclosporin immunosuppressive follow l...,immunosuppressive cyclosporin measure transpla...
2878277,3943554,1986,TAME stabilizes the cortex and mitotic apparat...,tame stabilizes the cortex and mitotic apparat...,tame stabilize the cortex and mitotic apparatu...,tame stabilize cortex mitotic apparatus sea ur...,stabilize mitotic urchin apparatus sea
2115072,3095978,1986,[Androgenic aspects of estrogens].,androgenic aspects of estrogens,androgenic aspect of estrogen,androgenic aspect estrogen,androgenic estrogen aspect


In [109]:
#overime title a jeho key words
print(data.iloc[0]['title'])
print(data.iloc[0]['tfidf_keywords'])

Deferred consent. A new approach for resuscitation research on comatose patients.
comatose consent resuscitation


In [110]:
#ulozime vyslednu tabulku do textoveho suboru
data.to_csv("1970/1970_90_p1.csv", sep = '\t', index = False)
#data.to_csv("1970/1970_90_p2.csv", sep = '\t', index = False)
#data.to_csv("1970/1970_90_p3.csv", sep = '\t', index = False)

In [111]:
#nacitame a zlepime 3tabulky
df1 = pd.read_csv("1970/1970_90_p1.csv", sep = '\t' ,on_bad_lines='skip')
df2 = pd.read_csv("1970/1970_90_p2.csv", sep = '\t' ,on_bad_lines='skip')
df3 = pd.read_csv("1970/1970_90_p3.csv", sep = '\t' ,on_bad_lines='skip')

df123 = pd.concat([df1, df2, df3])
df123

In [116]:
#ulozime jednu velku zlepenu tabulku
df123.to_csv("1970/1970_90.csv", sep = '\t', index = False)